<a href="https://colab.research.google.com/github/mjgpinheiro/Momentum-Trading-Example/blob/master/mean_reversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Trading Strategies"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "import bt\n",
    "import talib\n",
    "import matplotlib.pyplot as plt\n",
    "import pandas as pd"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Trading Signals"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Build an SMA-based signal strategy\n",
    "It's time to build and backtest your first signal-based strategy. Although simple, these types of strategies can be effective, and also lay the groundwork for more complex strategies consisting of additional signals and information.\n",
    "\n",
    "Implementing a price-comparison-based signal with bt is a straightforward process. You will first download some historical price data of the stock, calculate its SMA (simple moving average), implement an SMA-based signal strategy, and then backtest it with the stock price data."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get the price data\n",
    "price_data = bt.get('aapl', start='2019-11-1', end='2020-12-1')\n",
    "# Calculate the SMA\n",
    "sma = price_data.rolling(20).mean()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define the strategy\n",
    "bt_strategy = bt.Strategy('AboveSMA', \n",
    "                          [bt.algos.SelectWhere(price_data > sma),\n",
    "                           bt.algos.WeighEqually(),\n",
    "                           bt.algos.Rebalance()])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create the backtest and run it\n",
    "bt_backtest = bt.Backtest(bt_strategy, price_data)\n",
    "bt_result = bt.run(bt_backtest)\n",
    "# Plot the backtest result\n",
    "bt_result.plot(title='Backtest result')\n",
    "plt.savefig(\"Data/btresultsma.jpg\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<img src='Data/btresultsma.jpg' size=50>"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Build an EMA-based signal strategy\n",
    "Previously, you implemented an SMA-based signal strategy. However, you are wondering whether the EMA (exponential moving average) indicator is a better choice since it is more sensitive to recent price movement. You also would like to leverage the talib library to calculate the indicator. After switching to an EMA-based signal strategy, you will perform a similar backtest using the Apple stock price data."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate the EMA\n",
    "ema = talib.EMA(price_data['aapl'], timeperiod=20)\n",
    "# Convert to DataFrame\n",
    "ema = pd.DataFrame(ema)\n",
    "ema.rename(columns = {0:'aapl'}, inplace = True)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define the strategy\n",
    "bt_strategy = bt.Strategy('AboveEMA',\n",
    "                          [bt.algos.SelectWhere(price_data > ema),\n",
    "                           bt.algos.WeighEqually(),\n",
    "                           bt.algos.Rebalance()])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create the backtest and run it\n",
    "bt_backtest = bt.Backtest(bt_strategy, price_data)\n",
    "bt_result = bt.run(bt_backtest)\n",
    "# Plot the backtest result\n",
    "bt_result.plot(title='Backtest result')\n",
    "plt.savefig(\"Data/btresultema.jpg\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<img src='Data/btresultema.jpg' size=50>"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Trend-following Startegies"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get the price data\n",
    "price_data = bt.get('goog', start='2019-11-1', end='2020-12-1')\n",
    "# Calculate the SMA\n",
    "sma = price_data.rolling(20).mean()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate the indicators\n",
    "EMA_short = talib.EMA(price_data['goog'],\n",
    "                      timeperiod=10).to_frame()\n",
    "EMA_long = talib.EMA(price_data['goog'],\n",
    "                     timeperiod=40).to_frame()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create the signal DataFrame\n",
    "signal = EMA_long.copy()\n",
    "signal[EMA_long.isnull()] = 0"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Construct an EMA crossover signal\n",
    "Trend-following strategies believe that \"the trend is your friend,\" and use signals to indicate the trend and profit by riding it.\n",
    "\n",
    "You want to build and backtest a trend-following strategy. First, you decide to use two EMAs (exponential moving averages) to construct the signal. When the shorter-term EMA, EMA_short, is larger than the longer-term EMA, EMA_long, you will enter long positions in the market. Vice versa, when EMA_short is smaller than EMA_long, you will enter short positions."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Construct the signal\n",
    "signal[EMA_short > EMA_long] = 1\n",
    "signal[EMA_short < EMA_long] = -1"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Merge the data \n",
    "combined_df = bt.merge(signal, price_data, EMA_short, EMA_long)\n",
    "combined_df.columns = ['signal', 'Price', 'EMA_short', 'EMA_long']\n",
    "# Plot the signal, price and MAs\n",
    "combined_df.plot(secondary_y=['signal'])\n",
    "plt.savefig(\"Data/emacross.jpg\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<img src='Data/emacross.jpg' size=50>"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Build and backtest a trend-following strategy\n",
    "Previously, you constructed a signal using two EMA indicators. When the shorter-term EMA is larger than the longer-term EMA, the signal is 1 for entering long positions in the market. Vice versa, when the shorter-term EMA is smaller than the longer-term EMA, the signal is -1 for entering short positions. Now you will implement a trend-following strategy with your signal and perform a backtest using the Google stock."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [],
   "source": [
    "signal.rename(columns = {0:'goog'}, inplace = True)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define the strategy\n",
    "bt_strategy = bt.Strategy('EMA_crossover', \n",
    "                          [bt.algos.WeighTarget(signal),\n",
    "                           bt.algos.Rebalance()])\n",
    "\n",
    "# Create the backtest and run it\n",
    "bt_backtest = bt.Backtest(bt_strategy, price_data)\n",
    "bt_result = bt.run(bt_backtest)\n",
    "\n",
    "# Plot the backtest result\n",
    "bt_result.plot(title='Backtest result')\n",
    "plt.savefig(\"Data/btresultemacross.jpg\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<img src='Data/btresultemacross.jpg' size=50>"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Mean Reversion Strategy"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get the price data\n",
    "price_data = bt.get('goog', start='2019-11-1', end='2020-12-1')\n",
    "# Calculate the RSI\n",
    "stock_rsi = talib.RSI(price_data['goog']).to_frame()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create the same DataFrame structure as RSI\n",
    "signal = stock_rsi.copy()\n",
    "signal[stock_rsi.isnull()] = 0"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Construct an RSI based signal\n",
    "It's time to implement your first mean-reversion strategy. Mean reversion trading uses signals to detect market imbalance, and takes long positions in an oversold market and short positions in an overbought market.\n",
    "\n",
    "First, you will use the RSI indicator to gauge market conditions and construct the signal. If the RSI value drops below 30, you will enter long positions. If the RSI value rises above 70, you will enter short positions. If the RSI value is in between 30 and 70, you will take no positions."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Construct the signal\n",
    "signal[stock_rsi > 30] = -1\n",
    "signal[stock_rsi < 70] = 1\n",
    "signal[(stock_rsi <= 70) & (stock_rsi >= 30)] = 0\n",
    "\n",
    "# Plot the RSI\n",
    "stock_rsi.plot()\n",
    "plt.title('RSI')\n",
    "plt.savefig(\"Data/rsi.jpg\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<img src='Data/rsi.jpg' size=10>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Merge the data\n",
    "combined_df = bt.merge(signal, price_data)\n",
    "combined_df.columns = ['signal', 'Price']\n",
    "combined_df.plot(secondary_y =['signal'])\n",
    "plt.savefig(\"Data/rsisignal.jpg\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<img src='Data/rsisignal.jpg' size=50>"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Build and backtest a mean reversion strategy\n",
    "Previously, you constructed a signal using the RSI indicator. When the RSI value drops below 30, the signal is 1 for entering long positions in the market. When the RSI value rises above 70, the signal is -1 for entering short positions. Now you will implement a mean reversion strategy with the signal and perform a backtest on trading the Google stock."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "metadata": {},
   "outputs": [],
   "source": [
    "signal.rename(columns = {0:'goog'}, inplace = True)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define the strategy\n",
    "bt_strategy = bt.Strategy('RSI_MeanReversion', \n",
    "                          [bt.algos.WeighTarget(signal),\n",
    "                           bt.algos.Rebalance()])\n",
    "\n",
    "# Create the backtest and run it\n",
    "bt_backtest = bt.Backtest(bt_strategy, price_data)\n",
    "bt_result = bt.run(bt_backtest)\n",
    "# Plot the backtest result\n",
    "bt_result.plot(title='Backtest result')\n",
    "plt.savefig(\"Data/btresultrsi.jpg\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<img src='Data/btresultrsi.jpg' size=50>"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Strategy Optimization and Benchmarking"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Conduct a strategy optimization\n",
    "You have an SMA-based signal strategy to trade stocks. However, you are not sure what lookback period to use for calculating the SMA that can optimize the strategy performance. You plan to run multiple backtests on different input parameters. Also, you want the capability to assess the strategy on trading different stocks or based on different historical periods."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "metadata": {},
   "outputs": [],
   "source": [
    "def signal_strategy(ticker, period, name, start='2020-2-1', end='2020-11-1'):\n",
    "    # Get the data and calculate SMA\n",
    "    price_data = bt.get(ticker, start=start, end=end)\n",
    "    sma = price_data.rolling(period).mean()\n",
    "    # Define the signal-based trategy\n",
    "    bt_strategy = bt.Strategy(name,\n",
    "                              [bt.algos.SelectWhere(price_data > sma),\n",
    "                               bt.algos.WeighEqually(),\n",
    "                               bt.algos.Rebalance()])\n",
    "    # Return the backtest\n",
    "    return bt.Backtest(bt_strategy, price_data)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create signal strategy backtest\n",
    "ticker = 'tsla'\n",
    "sma10 = signal_strategy(ticker, period=10, name='SMA10')\n",
    "sma30 = signal_strategy(ticker, period=30, name='SMA30')\n",
    "sma50 = signal_strategy(ticker, period=50, name='SMA50')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Run all backtests and plot the resutls\n",
    "bt_results = bt.run(sma10, sma30, sma50)\n",
    "bt_results.plot(title='Strategy optimization')\n",
    "plt.savefig(\"Data/optimize-sma.jpg\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<img src='Data/optimize-sma.jpg' size=50>"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Perform a strategy benchmarking\n",
    "You are wondering: instead of spending energy actively trading a stock, what if you just sit back and hold the stock for a period of time. Does your active trading strategy generate better profits than a passive buy-and-hold strategy? To answer this question, you plan to perform a benchmarking test."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 24,
   "metadata": {},
   "outputs": [],
   "source": [
    "def buy_and_hold(ticker, name, start='2020-2-1', end='2020-11-1'):\n",
    "    # Get the data\n",
    "    price_data = bt.get(ticker, start=start, end=end)\n",
    "    # Define the benchmark strategy\n",
    "    bt_strategy = bt.Strategy(name, \n",
    "                              [bt.algos.RunOnce(),\n",
    "                               bt.algos.SelectAll(),\n",
    "                               bt.algos.WeighEqually(),\n",
    "                               bt.algos.Rebalance()])\n",
    "    # Return the backtest\n",
    "    return bt.Backtest(bt_strategy, price_data)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 25,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create benchmark strategy backtest\n",
    "benchmark = buy_and_hold('tsla', name='benchmark')\n",
    "\n",
    "# Run all backtests and plot the resutls\n",
    "bt_results = bt.run(sma10, sma30, sma50, benchmark)\n",
    "bt_results.plot(title='Strategy benchmarking')\n",
    "plt.savefig(\"Data/benchmark-sma.jpg\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<img src='Data/benchmark-sma.jpg' size=50>"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# Trading Strategies']},
  {'cell_type': 'code',
   'execution_count': 1,
   'metadata': {},
   'outputs': [],
   'source': ['import bt\n',
    'import talib\n',
    'import matplotlib.pyplot as plt\n',
    'import pandas as pd']},
  {'cell_type': 'markdown', 'metadata': {}, 'source': ['## Trading Signals']},
  {'cell_type': 'markdown',
   'metadata': {},
   'source': ['### Build an SMA-based signal strategy\n',
    "It's time to build and backtest your first signal-based strategy. Although simple, these types of strategies can be effective, and also lay the groundwork for more complex strategies consisting of additional signals and information.\n",
    '\n',
    'Implementing a price-comparison-based signal with bt is a straightforward process. You will first download some historical price data of the stock, calculate its SMA (simple moving average), implement an SMA-based signal strategy, and then backtest it with